In [3]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, MinMaxScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, mean_absolute_error

from sklearn import preprocessing
from sklearn import cluster
from sklearn.cluster import KMeans

In [5]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [7]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [6]:
portfolio.groupby('offer_type')['id'].count()

offer_type
bogo             4
discount         4
informational    2
Name: id, dtype: int64

In [8]:
received = transcript.query(" event == 'offer received' ")['time'].mean()
completed = transcript.query(" event == 'offer completed' ")['time'].mean()
viewed = transcript.query(" event == 'offer viewed' ")['time'].mean()
transactions = transcript.query(" event == 'transaction' ")['time'].mean()

print('mean time of offer received:', received)
print('mean time of offer viewed:', viewed)
print('mean time of offer completed:', completed)
print('mean time of transactions:', transactions)

mean time of offer received: 332.57951938330035
mean time of offer viewed: 354.29051537462107
mean time of offer completed: 401.0528008576789
mean time of transactions: 381.58433427130035


In [9]:
received = transcript.query(" event == 'offer received' ")['time'].count()
completed = transcript.query(" event == 'offer completed' ")['time'].count()
viewed = transcript.query(" event == 'offer viewed' ")['time'].count()
transactions = transcript.query(" event == 'transaction' ")['time'].count()

print('offer received counts:', received)
print('offer viewed counts:', viewed)
print('offer completed counts:', completed)
print('transactions counts:', transactions)

offer received counts: 76277
offer viewed counts: 57725
offer completed counts: 33579
transactions counts: 138953


In [10]:
transcript.query(" event == 'transaction' ")

,person,event,value,time
12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,{'amount': 34.56},0
12659,54890f68699049c2a04d415abc25e717,transaction,{'amount': 13.23},0
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,{'amount': 19.51},0
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,{'amount': 18.97},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


## Data Pre-processing:

In [11]:
portfolio.rename(columns = {'id': 'offer_id'}, inplace = True)